# Aula 8 - Introducao ao Pandas e Redes Neurais 

Nesta aula vamos ver como utilizar um arquivo ***.csv*** para receber um conjunto de dados e utiliza-los como entradas na nossa rede neural que estamos comecando a construir. Comecamos importando as bibliotecas que vamos utilizar

In [1]:
import pandas as pd ## para ler o arquivo com os dados
import numpy as np ## para fazer computacoes matematicas

O arquivo que vamos utilizar tem o nome de ***ripley.csv*** e se encontra disponivel na mesma pasta que este notebook. Nele vamos encontrar 1250 linhas, cada uma contendo tres informacoes, a coordenada ***x1***, a coordenada ***x2*** e o alvo, ou seja, a classe ***y*** que pode assumir os valores de 0 ou 1. E importante ainda depois que recebermos os nossos dados, mistura-los, pois eles estao praticamente separados entre zeros e uns.

**Obs**: df e uma abreviacao para ***DataFrame*** que e a classe do objeto retornado pela funcao ***read_csv***

In [2]:
df = pd.read_csv('ripley.csv') ## recebendo o conjunto de dados
df = df.sample(frac=1).reset_index(drop=True) ## misturando os dados do nosso conjunto

In [3]:
df.head()

,x1,x2,y
0,-0.439950,0.520993,1.0
1,-0.448940,0.650604,1.0
2,-0.406753,0.858976,1.0
3,-0.294967,0.707617,1.0
4,0.381000,0.653581,1.0


A funcao ***.head()*** nos permite olhar as primeiras cinco linhas do nosso conjunto. Veja que alem dos nomes das colunas, x1, x2 e y recebemos tambem para cada linha um **id** representado pelo primeiro numero em negrito, este valor nos permite acessar cada linha quando quisermos utilizando a sintaxe ***.iloc[id]***

In [4]:
print(df.shape) ## checando o tamanho e dimensao do nosso conjunto
df_treino = df.iloc[:750] ## determinando os dados para treinamento
df_val = df.iloc[750:1000] ## determinando os dados para validacao
df_teste = df.iloc[1000:] ## determinando os dados para teste

(1250, 3)


Em redes neurais trabalhamos geralmente com tres conjuntos de dados.

1. ***Treino***: e o conjunto que vamos utilizar para treinar a nossa rede neural, assim como treinamos nosso regressor linear nas outras aulas.
1. ***Validacao***: e o conjunto que vamos utilizar para checar quao boa nossa rede, ou seja, vamos checar quantos dados do conjunto de validacao nossa rede consegue estimar corretamente.
1. ***Teste***: conjunto para os quais nao conhecemos o alvo, y, e queremos estima-los com nossa rede

Note que apesar de que aqui nos conhecemos os **y** para os dados no conjunto de Teste, estamos utilizando essa notacao para nos familiarizarmos com as ideias.

A seguir vamos receber agora os valores de nossos conjuntos, a propriedade **values** retorna uma matriz em **numpy** com os valores numericos de nossos conjuntos. 

**Breve explicacao da indexagem(??)**: Em python quando utilizamos -1 para acessar algum index, estamos na realidade acessando o ulitmo index, entao, se tivermos um vetor **v** = [0, 2, 4, 6, 8], entao v[-1] = 8, v[-2] = 6 e assim por diante. Como este assunto foi coberto em outras aulas vamos continuar.

In [6]:
x_treino, y_treino = df_treino.iloc[:, :-1].values, df_treino.iloc[:,-1].values
x_val, y_val = df_val.iloc[:, :-1].values, df_val.iloc[:,-1].values
x_teste, y_teste = df_teste.iloc[:, :-1].values, df_teste.iloc[:,-1].values

In [7]:
x_treino.shape, y_treino.shape

((750, 2), (750,))

Certo, agora que temos os conjuntos vamos comecar escrevendo nossa rede neural, na [Aula 6](https://github.com/israelcamp/AulasPython/tree/master/Aula6) coloquei links sobre redes neurais, nas aulas presenciais tambem foi explicado como elas funcionam.

Vamos comecar definindo alguns parametros da nossa rede

In [8]:
n = 2 ## dimensao do nosso vetor de entrada (x1, x2)
nos = 10 ## quantidade de nos escondidos da nossa rede neural
nc = 1 ## quantidade de nos de saida

Como so temos duas classes possiveis para cada amostra, 0 ou 1, vamos utilizar somente um no de saida. A seguir vamos inicializar aletoriamente as matrizes que guardarao os pesos de nossa rede. 

In [9]:
W = np.random.rand(n, nos) ## matriz que guarda os vetores da camada de entrada para a camada escondida
b = np.random.rand(nos) ## vetor que ira guardar os biases da camada de entrada ate a camada escondida
V = np.random.rand(nos, nc) ## matriz, neste caso vetor, que guarda os pesos da camada escondida para a de saida

Com as matrizes vamos escrever uma funcao que dado a vetor de entrada, amostra, nos devolve a predica da nossa rede

In [10]:
def predict(x):
    S = np.dot(x, W) + b
    return np.dot(S, V)

Vamos checar que esta funcionando passando a primeira linha de nossa matriz de treino

In [12]:
yb = predict(x_treino[0])
print(f'Predicao {yb}, para alvo {y_treino[0]}')

Predicao [2.92214074], para alvo 1.0


Funciona e apesar de estarmos longe do valor real 1, nossa rede na verdade classificou corretamente esta amostra, pois como vimos na [Aula 7](https://github.com/israelcamp/AulasPython/tree/master/Aula7), vamos considerar valores maiores que 0.5 como pertencentes a classe 1 e menor que 0.5 a classe 0. Vamos entao escrever uma funcao que nos diga o quao correta nossa rede e para algumas amostras, neste caso vamos utilizar o conjunto de validacao.

In [13]:
def acuracia(x, y):
    yb = predict(x)
    yb = 1. * (yb > 0.5)
    return sum(yb == y)/len(y)

Vamos parar um pouco para entender o que esta acontecendo dentro desta funcao. Primeiramente, como vimos **predict** vai nos retornar a predicao para o **x** dado, note que predict pode receber tanto vetores quanto matrizes.

In [14]:
print(predict(x_val[0]).shape, predict(x_val).shape)

(1,) (250, 1)


Segundo, como vimos os valores de **y** que vem dos nossos conjuntos assumem somente os valores de 0 e 1, logo, precisamos converter o valor retornado por predict para 0 ou 1 utilizando 0.5 como threshold.

In [16]:
algunsy = predict(x_val[:10]) ## estimando 10 linhas
print('Valores estimandos \n', algunsy)
print('Quais sao maiores que 0.5? \n', algunsy > 0.5)
algunsy = 1 * (algunsy > 0.5)
print('Classificando os estimados\n',algunsy)

Valores estimandos 
 [[ 6.23735367]
 [ 3.9896492 ]
 [ 1.40702917]
 [ 4.54129165]
 [-0.07080787]
 [ 4.05792792]
 [ 4.47505396]
 [ 6.10900086]
 [ 4.93566326]
 [ 2.34905737]]
Quais sao maiores que 0.5? 
 [[ True]
 [ True]
 [ True]
 [ True]
 [False]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]]
Classificando os estimados
 [[1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]]


Vamos entao receber a acuracia da nossa rede no conjunto de validacao

In [18]:
print('Acuracia na validacao', acuracia(x_val, y_val.reshape(-1,1)))

Acuracia na validacao [0.552]


O **reshape** em y_val e necessario pois o valor retornado por predict tem dimensao diferente de y_val (por que?), logo reshape nos permite torna-los iguais. 

In [19]:
??np.reshape

Vamos terminar esta aula por aqui, na proxima vamos finalmente utilizar os dados no conjunto de treino para treinar nossa rede neural.